# generate feature list
```
python3 gen_feat.py --inf_list img/img.list --feat_list img/feat.list --resume magface_epoch_00025.pth
```

In [ ]:
import numpy as np
import math
import cv2
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="white") 
%matplotlib inline

# Visualize magnitudes (qualities)

In [ ]:
def imshow(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
def show(idx_):
    imgname = imgnames[idx_]
    img = cv2.imread(imgname)
    imshow(img)
    print(img_2_mag[imgname], imgname)    

In [ ]:
with open('img/feat.list', 'r') as f:
    lines = f.readlines()

In [ ]:
img_2_feats = {}
img_2_mag = {}
for line in lines:
    parts = line.strip().split(' ')
    imgname = parts[0]
    feats = [float(e) for e in parts[1:]]
    mag = np.linalg.norm(feats)
    img_2_feats[imgname] = feats/mag
    img_2_mag[imgname] = mag

In [ ]:
imgnames = list(img_2_mag.keys())
mags = [img_2_mag[imgname] for imgname in imgnames]
sort_idx = np.argsort(mags)

In [ ]:
H, W = 112, 112
NW = 9
# Calcola automaticamente il numero di righe in base al numero di immagini
NH = math.ceil(108 / NW)
canvas = np.zeros((NH * H, NW * W, 3), np.uint8)

for i, ele in enumerate(sort_idx):
    imgname = '/'.join(imgnames[ele].split('/')[-2:])
    img = cv2.imread(imgname)
    canvas[int(i / NW) * H: (int(i / NW) + 1) * H, (i % NW) * W: ((i % NW) + 1) * W, :] = img    

In [ ]:
plt.figure(figsize=(20, 20))
print([float('{0:.2f}'.format(mags[idx_])) for idx_ in sort_idx])
imshow(canvas)

# visualize recognition

In [ ]:
feats = np.array([img_2_feats[imgnames[ele]] for ele in sort_idx])
sim_mat = np.dot(feats, feats.T)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))  # Creazione della figura per la heatmap
sns.heatmap(sim_mat, cmap="coolwarm", annot=False, cbar=True, linewidths=0.5, ax=ax)  # Mappa di calore senza annotazioni

# Aggiungi un titolo
ax.set_title("Matrice di Similarità", fontsize=16)
plt.show()